In [413]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import numpy as np
import time
import os
import pandas as pd
from unicodedata import normalize
import textract
import re
from time import sleep
from fake_useragent import UserAgent
import pandas as pd
from tqdm import tqdm
import numpy as np

Ezt az oldalt fogjuk scrapelni:
<br>
https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium

# Alapok

## Request

In [414]:
URL = "https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium"

In [415]:
resp = requests.get(URL)

In [416]:
type(resp)

requests.models.Response

In [417]:
resp.status_code

200

In [419]:
resp.raise_for_status()

In [420]:
resp.content

b'<!DOCTYPE html>\n<html lang="hu">\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta charset="utf-8">\n<title>Szakkoll\xc3\xa9gium - Gyakori k\xc3\xa9rd\xc3\xa9sek</title>\n<meta name="description" content="Szakkoll\xc3\xa9gium - K\xc3\xa9rd\xc3\xa9sek a t\xc3\xa9m\xc3\xa1ban. Pl. A budapesti szakkoll\xc3\xa9giumokba csak akkor jelentkezhetek ha az adott egyetemre j\xc3\xa1rok?">\n<meta name="keywords" content="szakkoll\xc3\xa9gium, k\xc3\xa9rd\xc3\xa9sek, v\xc3\xa1laszok">\n<meta property="og:title" content="Szakkoll\xc3\xa9gium - Gyakori k\xc3\xa9rd\xc3\xa9sek">\n<meta property="og:description" content="Szakkoll\xc3\xa9gium - K\xc3\xa9rd\xc3\xa9sek a t\xc3\xa9m\xc3\xa1ban. Pl. A budapesti szakkoll\xc3\xa9giumokba csak akkor jelentkezhetek ha az adott egyetemre j\xc3\xa1rok?">\n<meta property="og:type" content="website">\n<meta property="og:image" content="//static.gyakorikerdesek.hu/p/logo.jpg">\n<link rel="shortcut icon" href="//static.gyakorikerd

## Keresés a DOM-ban

In [421]:
soup = BeautifulSoup(resp.content, "html5lib")

In [422]:
type(soup)

bs4.BeautifulSoup

In [423]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="hu">
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta charset="utf-8"/>
  <title>
   Szakkollégium - Gyakori kérdések
  </title>
  <meta content="Szakkollégium - Kérdések a témában. Pl. A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?" name="description"/>
  <meta content="szakkollégium, kérdések, válaszok" name="keywords"/>
  <meta content="Szakkollégium - Gyakori kérdések" property="og:title"/>
  <meta content="Szakkollégium - Kérdések a témában. Pl. A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?" property="og:description"/>
  <meta content="website" property="og:type"/>
  <meta content="//static.gyakorikerdesek.hu/p/logo.jpg" property="og:image"/>
  <link href="//static.gyakorikerdesek.hu/favicon.ico" rel="shortcut icon"/>
  <link href="//static.gyakorikerdesek.hu/apple-touch-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
  <meta co

Látjuk hogy vannak `div` típusú elemek `kerdeslista` class-el, amelyek az egyes kérdéseket lehatárolják. 

*Lásd Chrome inspect funkció.*

### Egy elem kijelölése

Jelöljük ki az első ilyen elemet

In [424]:
elem = soup.find("div", attrs={"class", "kerdeslista"})

In [426]:
print(elem.prettify())

<div class="flex kerdeslista" id="k12165689" style="flex-wrap: nowrap;">
 <div class="kerdes_normal">
 </div>
 <div class="kerdeslista_szoveg">
  <a href="/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja">
   A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?
  </a>
  <br/>
  <span class="kerdes_alatt">
   <a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__egyeb-kerdesek">
    Közoktatás, tanfolyamok » Egyéb kérdések
   </a>
  </span>
 </div>
 <div class="kerdeslista_valasz">
  1
 </div>
</div>



A `prettify` method segít szépen ábrázolni a DOM-ot.

In [427]:
soup.find("div", attrs={"class": "kerdeslistak"})

In [428]:
print(elem.prettify())

<div class="flex kerdeslista" id="k12165689" style="flex-wrap: nowrap;">
 <div class="kerdes_normal">
 </div>
 <div class="kerdeslista_szoveg">
  <a href="/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja">
   A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?
  </a>
  <br/>
  <span class="kerdes_alatt">
   <a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__egyeb-kerdesek">
    Közoktatás, tanfolyamok » Egyéb kérdések
   </a>
  </span>
 </div>
 <div class="kerdeslista_valasz">
  1
 </div>
</div>



A fenti kijelölést egyébként máshogy is meg lehet csinálni, erre íme pár példa

In [200]:
soup.find("div", class_="kerdeslista")

<div class="flex kerdeslista" id="k12165689" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja">A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__egyeb-kerdesek">Közoktatás, tanfolyamok » Egyéb kérdések</a></span></div><div class="kerdeslista_valasz">1</div></div>

In [429]:
soup.select_one("div.kerdeslista")

<div class="flex kerdeslista" id="k12165689" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja">A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__egyeb-kerdesek">Közoktatás, tanfolyamok » Egyéb kérdések</a></span></div><div class="kerdeslista_valasz">1</div></div>

Ez utóbbi CSS selector alapján keres. Szerintem ez a legjobb módszer egyébként, univerzálisabban használható.

Segítség hozzá itt:  
https://www.w3schools.com/cssref/css_selectors.php

### Több elem kijelölése

Hasonló az előzőhöz, csak egy listát kapunk vissza a patternnek megfelelő objektumok alapján.

Standard `Beautifulsoup` megoldás

In [202]:
elems = soup.find_all("div", attrs={"class", "kerdeslista"})

In [430]:
elems[0]

<div class="kerdes"><div class="kerdes_fejlec"><h1>Miért nem veszik fel az ELTE btk.-n a telefont?</h1>
</div><div class="kerdeslista_ikon"></div><div class="kerdes_kerdes"><div style="width: 100%; margin-bottom: 10px; text-align: right;"><input class="check" id="chkb" onclick='location.href="/belepes"; return false;' type="checkbox"/> Figyelt kérdés</div>Napok óta próbálom hívni őket, hogy a halasztási lehetőségekről érdeklődjek, illetve a szakkollégiumi kérdéseim is lennének. Erre csicsk4 diszpécser csaj beleszól unottan, hogy "JOOOL VAAAAN TOVÁBB KAPCSOLOM", ásít, tovább kapcsol, aztán kicsöng és semmi, ez volt az első alkalom, másodszorra már fel se veszi, mármint a ügyintéző kiscsaj, biztos letiltotta a számom. Miért mindenki hülye ebben az országban? Rendesen kezdek undorodni az ELTE-től. Normálisan szeretnék érdeklődni és nem lehet, azóta ahová átirányított is próbáltam felhívni azzal a mellékkel, de sose veszik fel, egyszer sem. Pedig annál az egy alkalomnál mikor méltóztatott 

In [431]:
len(elems)

6

CSS selectorral

In [432]:
soup.select("div.kerdeslista")

[<div class="flex kerdeslista" id="k12165689" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja">A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__egyeb-kerdesek">Közoktatás, tanfolyamok » Egyéb kérdések</a></span></div><div class="kerdeslista_valasz">1</div></div>,
 <div class="flex kerdeslista sor" id="k12143277" style="flex-wrap: nowrap;"><div class="kerdes_normal"></div><div class="kerdeslista_szoveg"><a href="/kozoktatas-tanfolyamok__magyar-iskolak__12143277-miert-nem-veszik-fel-az-elte-btk-n-a-telefont">Miért nem veszik fel az ELTE btk.-n a telefont?</a><br/><span class="kerdes_alatt"><a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__magyar-iskolak">Közoktatás, tanfolyamok 

# Scraping

## Linkek begyűjtése

Ahhoz hogy be tudjuk gyűjteni a hozzászólásokat, először meg kéne tudni mondani, hogy pontosan hány ilyen kérdés volt és azokat milyen linken lehet elérni.

**Segítség:**  
Ilyenkor elég végig iterálni az oldalakon és begyűjteni minden linket!

Ha megnyitod a második oldalt, akkor látszik, hogy az URL az alábbira változik:  
https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium&oldal=2

Tehát ebből már sejthető, hogy az URL végén az `oldal=2` mondja majd meg, hogy hanyadik oldalon vagyunk jelenleg. Ez egyébként gyakorta így van, mivel az úgynevezett [query stringet](https://en.wikipedia.org/wiki/Query_string) hasznják a legtöbbször.

A query stringek a `request.get` methodnál könnyen megadhatóak egy dictionary-vel:

In [434]:
resp = requests.get(url=URL, params={"oldal": 2})

In [435]:
resp.url

'https://www.gyakorikerdesek.hu/kereses.php?keres=szakkoll%C3%A9gium&oldal=2'

Az egyszerűség kedvéért itt egy függvényt, ami a requestet és a parseolást egyben elvégzi.

In [436]:
def get_soup(url: str, params: dict = None):
    resp = requests.get(url=url, params=params)
    resp.raise_for_status()
    return BeautifulSoup(resp.content, "html5lib")

### Oldalak száma

Már csak azt kéne megtudni, hogy összesen hány oldal lehet.

Ha ránézel az oldalra, látod, hogy gombokkal lehet váltani oldalakat. Ez szuper, mert akkor valószínűleg a HTML-ben is nyoma van ennek. Még látványosabb, hogy az utolsó gomb mindig az utolsó oldalra vezet, tehát nekünk erre lesz majd szükségünk.

Ezt az utolsó elemet egy `div` tárolja, aminek `oldalszamok` a class-e. Ezen belül pedig `a` elemek reprezentálják a gombokat.

In [438]:
soup.select("div.oldalszamok a")

[<a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=2" title="szakkollégium 2.">2</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=3" title="szakkollégium 3.">3</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=4" title="szakkollégium 4.">4</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=5" title="szakkollégium 5.">5</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=6" title="szakkollégium 6.">6</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=7" title="szakkollégium 7.">7</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=8" title="szakkollégium 8.">8</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=9" title="szakkollégium 9.">9</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=10" title="szakkollégium 10.">10</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=2" title="szakkollégium 2.">❯</a>,
 <a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=10" title

Látszik, hogy a tetején és az alján is vannak ilyen gombok, de ez nem gond, mert az utolsó elég lesz.

In [439]:
elem = soup.select("div.oldalszamok a")[-1]
elem

<a href="/kereses.php?keres=szakkoll%C3%A9gium&amp;oldal=10" title="szakkollégium 10.">❯❯</a>

Ez az az elem, most szedjük ki belőle a releváns információt. A `href` attribútum fogja ez tárolni egy link formájában.

Ezt a `soup.get` methoddal lehet kiszedni.

In [440]:
last_url = elem.get("href")
last_url

'/kereses.php?keres=szakkoll%C3%A9gium&oldal=10'

Végül szedjük ki a string végéről a számot

In [47]:
int(last_url.split("=")[-1])

10

Az egész egy függvényben:

In [441]:
def get_last_page(url: str):
    soup = get_soup(url=url)
    last_elem = elem = soup.select("div.oldalszamok a")[-1]
    last_url = last_elem.get("href")
    return int(last_url.split("=")[-1])

In [442]:
get_last_page(url=URL)

10

### URL tábla összeállítása

Korábban már láttuk, hogy így néz ki egy kérdés

In [443]:
elem = soup.select("div.kerdeslista")[0]
print(elem.prettify())

<div class="flex kerdeslista" id="k12165689" style="flex-wrap: nowrap;">
 <div class="kerdes_normal">
 </div>
 <div class="kerdeslista_szoveg">
  <a href="/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja">
   A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?
  </a>
  <br/>
  <span class="kerdes_alatt">
   <a class="kerdes_alatt" href="/kozoktatas-tanfolyamok__egyeb-kerdesek">
    Közoktatás, tanfolyamok » Egyéb kérdések
   </a>
  </span>
 </div>
 <div class="kerdeslista_valasz">
  1
 </div>
</div>



Nekünk ebből az **id**, a **szöveg** és az **url** fog kelleni.

In [444]:
elem.get("id")

'k12165689'

In [445]:
elem.select_one("div.kerdeslista_szoveg a").get("href")

'/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja'

A szöveget a `soup.get_text` methoddal tudjátok kinyerni. A strip paraméter megadja, hogy levágja-e a felesleges karaktereket a string két oldaláról.

In [446]:
elem.select_one("div.kerdeslista_szoveg a").get_text(strip=True)

'A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?'

Egyben az egész egy függvényként

In [447]:
def _parse_listing_elem(elem):
    return {
        "id": elem.get("id"),
        "url": elem.select_one("div.kerdeslista_szoveg a").get("href"),
        "question": elem.select_one("div.kerdeslista_szoveg a").get_text(strip=True),
    }

In [448]:
_parse_listing_elem(elem)

{'id': 'k12165689',
 'url': '/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja',
 'question': 'A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?'}

Most csináljunk egy függvényt egy oldal összers kérdésének kinyerésére is.

In [449]:
def parse_listing_page(soup):
    return [_parse_listing_elem(elem) for elem in soup.select("div.kerdeslista")]

In [450]:
parse_listing_page(soup)[:5]

[{'id': 'k12165689',
  'url': '/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja',
  'question': 'A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?'},
 {'id': 'k12143277',
  'url': '/kozoktatas-tanfolyamok__magyar-iskolak__12143277-miert-nem-veszik-fel-az-elte-btk-n-a-telefont',
  'question': 'Miért nem veszik fel az ELTE btk.-n a telefont?'},
 {'id': 'k12104749',
  'url': '/kozoktatas-tanfolyamok__egyeb-kerdesek__12104749-szakkollegiumi-tapasztalatok',
  'question': 'Szakkollégiumi tapasztalatok?'},
 {'id': 'k12090303',
  'url': '/kozoktatas-tanfolyamok__tanulasi-lehetosegek__12090303-mit-kell-tudni-az-egyetemi-szakkollegiumokrol-pontosan-hogy-mukodik-mik-a-tags',
  'question': 'Mit kell tudni az egyetemi szakkollégiumokról? pontosan hogy működik, mik a tagsági feltételek, és mik kötelezettségek?'},
 {'id': 'k11913351',
  'url': '/kozoktatas-tanfolyamok__magyar-iskolak__

És végül így már végig tudunk iterálni az összes oldalon.

In [453]:
pages = list(range(1, get_last_page(url=URL) + 1))
pages

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [454]:
listing_recs = []
for page in tqdm(pages):
    soup = get_soup(url=URL, params={"oldal": page})
    listing_recs.extend(parse_listing_page(soup=soup))

100%|██████████| 10/10 [00:01<00:00,  6.63it/s]


Beolvasás `DataFrame`-be.

In [456]:
df_listing = pd.DataFrame(listing_recs)

In [457]:
df_listing.head(5)

,id,url,question
0,k12165689,/kozoktatas-tanfolyamok__egyeb-kerdesek__12165...,A budapesti szakkollégiumokba csak akkor jelen...
1,k12143277,/kozoktatas-tanfolyamok__magyar-iskolak__12143...,Miért nem veszik fel az ELTE btk.-n a telefont?
2,k12104749,/kozoktatas-tanfolyamok__egyeb-kerdesek__12104...,Szakkollégiumi tapasztalatok?
3,k12090303,/kozoktatas-tanfolyamok__tanulasi-lehetosegek_...,Mit kell tudni az egyetemi szakkollégiumokról?...
4,k11913351,/kozoktatas-tanfolyamok__magyar-iskolak__11913...,PhD-sként késő szakkolégiumba jelentkezni?


## Szövegek kiszedése

In [458]:
listing_recs[0]["url"]

'/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja'

Ilyen URL-eket kaptunk a korábbi lépésben. Ezeket úgy kell értelmezni, hogy az alap URL (https://www.gyakorikerdesek.hu) utáni részt jelölik:
https://www.gyakorikerdesek.hu/kozoktatas-tanfolyamok__magyar-iskolak__11913351-phd-skent-keso-szakkolegiumba-jelentkezni

In [460]:
BASE_URL = "https://www.gyakorikerdesek.hu"
BASE_URL + listing_recs[0]["url"]

'https://www.gyakorikerdesek.hu/kozoktatas-tanfolyamok__egyeb-kerdesek__12165689-a-budapesti-szakkollegiumokba-csak-akkor-jelentkezhetek-ha-az-adott-egyetemre-ja'

### Parsing függvények

Nézzünk meg jobban egy oldalt. A szövegeket a `kerdes_kerdes` vagy `valasz_valasz` class-el rendelkező `div` elemek tartalmazzák.

In [463]:
soup = get_soup(url=BASE_URL + listing_recs[1]["url"])

Válasszuk ki a megfelelő elemeket. A CSS selectorban a vessző az uniót jelöli.

In [464]:
elems = soup.select("div.kerdes, div.valasz")

Nézzünk meg egy ilyen elemet.

In [465]:
print(elems[1].prettify())

<div class="valasz" id="valasz-75983453">
 <div class="valasz_fejlec">
  1/4
  <span class="anonim">
   anonim
  </span>
  <span class="vsz">
   <img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/>
   <img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/>
   <img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/>
   <img alt="*" src="//static.gyakorikerdesek.hu/p/vsz2.png"/>
   <img alt="*" src="//static.gyakorikerdesek.hu/p/vsz1.png"/>
  </span>
  válasza:
 </div>
 <div class="valasz_valasz">
  <div class="flex" style="flex-wrap: nowrap;">
   <div id="ertek75983453" style="margin-right: 10px;">
    <svg class="ertek" viewBox="0 0 100 100">
     <circle cx="50" cy="50" fill="#C4DCC5" r="45">
     </circle>
     <path d="M50 10 a 40 40 0 0 1 0 80 a 40 40 0 0 1 0 -80" fill="none" stroke="#1F8D25" stroke-dasharray="251.2,0" stroke-linecap="round" stroke-width="7">
     </path>
     <text dy="7" font-size="20" text-anchor="middle" x="50" y="50">
      100%
     </text>
    

In [466]:
def parse_msg_text(soup: BeautifulSoup):
    elem = soup.select_one("div.kerdes_kerdes, div.valasz_valasz")
    return elem.get_text(separator="\n")


parse_msg_text(elems[1])

'100%\nMert ezen a héten még tavaszi szünet van, és a tanulmányi hivatalban csak ügyelet van. Írj nekik e-mailt.\n \n[link]\n '

In [468]:
def parse_msg_date(soup):
    return soup.select_one("div.kerdes_statusz div, div.valasz_statusz div").get_text()


parse_msg_date(elems[1])

'febr. 20. 18:22'

In [470]:
def parse_msg_header(soup):
    return soup.select_one("div.kerdes_fejlec, div.valasz_fejlec").get_text()


parse_msg_header(elems[1])

'\xa01/4 anonim  válasza:'

Végül írjunk egy függvényt, ami feldolgoz egy kommentet.

In [472]:
def parse_msg(elem):
    return {
        "comment": parse_msg_text(elem),
        "date": parse_msg_date(elem),
        "header": parse_msg_header(elem),
    }


parse_msg(elems[1])

{'comment': '100%\nMert ezen a héten még tavaszi szünet van, és a tanulmányi hivatalban csak ügyelet van. Írj nekik e-mailt.\n \n[link]\n ',
 'date': 'febr. 20. 18:22',
 'header': '\xa01/4 anonim  válasza:'}

És egy olyat is, ami az adott oldal összes kommentjét feldolgozza.

In [473]:
def parse_all_msg(soup):
    elems = soup.select("div.kerdes, div.valasz")
    return [parse_msg(e) for e in elems if "resv" not in e.get("class")]


parse_all_msg(soup)

[{'comment': ' Figyelt kérdés\nNapok óta próbálom hívni őket, hogy a halasztási lehetőségekről érdeklődjek, illetve a szakkollégiumi kérdéseim is lennének. Erre csicsk4 diszpécser csaj beleszól unottan, hogy "JOOOL VAAAAN TOVÁBB KAPCSOLOM", ásít, tovább kapcsol, aztán kicsöng és semmi, ez volt az első alkalom, másodszorra már fel se veszi, mármint a ügyintéző kiscsaj, biztos letiltotta a számom. Miért mindenki hülye ebben az országban? Rendesen kezdek undorodni az ELTE-től. Normálisan szeretnék érdeklődni és nem lehet, azóta ahová átirányított is próbáltam felhívni azzal a mellékkel, de sose veszik fel, egyszer sem. Pedig annál az egy alkalomnál mikor méltóztatott a kiscsaj felvenni, tök kedves és normális voltam, ő meg ok nélkül csicsk4 az elejétől kezdve? Azt hiszem megb4sztatom a dékánt, aztán pedig a rektort, a piszlicsáré, jelentéktelen messze az ő szintjük alatti kérdéseimmel. Igen, ez lesz a helyes út.',
  'date': 'febr. 20. 18:13',
  'header': 'Miért nem veszik fel az ELTE btk.

### Összes oldal feldolgozása

Végül iteráljunk végig az összes thread-en és szedjük ki a kommenteket.

Az id-k miatt `DataFrame`-et használunk. Erről később lesz szó.

In [474]:
comments = []
for listing in tqdm(listing_recs):
    soup = get_soup(url=BASE_URL + listing["url"])
    comment_recs = parse_all_msg(soup)
    comments.append(pd.DataFrame(comment_recs).assign(thread_id=listing["id"]))

100%|██████████| 60/60 [00:15<00:00,  3.78it/s]


In [475]:
df_comments = pd.concat(comments)
df_comments

,comment,date,header,thread_id
0,Figyelt kérdés\nPéldául az ELTE szakkollégium...,márc. 9. 14:52,A budapesti szakkollégiumokba csak akkor jelen...,k12165689
1,Ha csak az elteseknek hirdetik meg a tagfelvét...,márc. 9. 14:55,1/1 anonim válasza:,k12165689
0,Figyelt kérdés\nNapok óta próbálom hívni őket...,febr. 20. 18:13,Miért nem veszik fel az ELTE btk.-n a telefont?\n,k12143277
1,100%\nMert ezen a héten még tavaszi szünet van...,febr. 20. 18:22,1/4 anonim válasza:,k12143277
2,65%\nMert még nem kezdődött meg a félév...,febr. 20. 18:31,2/4 anonim válasza:,k12143277
...,...,...,...,...
3,annak a szaknak a kimondott kolesza ahova felv...,2010. júl. 28. 18:49,3/5 anonim válasza:,k1039145
4,"Nem, nem azt jelenti. A szakkollégium név arra...",2010. júl. 28. 19:04,4/5 anonim válasza:,k1039145
5,Na de akkor csak abba a koleszba felvételizhet...,2010. júl. 29. 21:41,5/5 A kérdező kommentje:,k1039145
0,"Figyelt kérdés\nLakott már vki ott, és milyen...",2009. júl. 18. 20:58,Budapesti Márton Áron Szakkollégium milyen hel...,k327099


Normalizálás

In [479]:
df_comments["header"].iloc[1]

'\xa01/1 anonim  válasza:'

In [335]:
df_comments["header"].str.normalize("NFKD").iloc[1]

' 1/1 anonim  válasza:'

In [481]:
df_comments = df_comments.assign(
    header=lambda _df: _df["header"].str.normalize("NFKD"),
    comment=lambda _df: _df["comment"].str.normalize("NFKD"),
)

Regex

In [349]:
df_comments["comment"]

0     Figyelt kérdés\nPéldául az ELTE szakkolle...
1    Ha csak az elteseknek hirdetik meg a tagfelvé...
0     Figyelt kérdés\nNapok óta próbálom hívn...
1    100%\nMert ezen a héten még tavaszi szünet ...
2      65%\nMert még nem kezdődött meg a félév...
                           ...                        
0     Figyelt kérdés\nNagyon keveset tudok róla,...
1                                    83%\n \n[link]\n 
0     Figyelt kérdés\nAz első két évemben a vi...
0     Figyelt kérdés\nHeller, Rajk, SzISZ, esetle...
1    84%\nMCC, ha nem zavar az agymosásos része, ...
Name: comment, Length: 83, dtype: object

In [483]:
rate_rgx = re.compile("(?P<rate>\d{1,3})?%?(?P<text>.*)", re.DOTALL)
df_comments["comment"].str.extract(rate_rgx)

,rate,text
0,NaN,Figyelt kérdés\nPéldául az ELTE szakkolle...
1,NaN,Ha csak az elteseknek hirdetik meg a tagfelvé...
0,NaN,Figyelt kérdés\nNapok óta próbálom hívn...
1,100,"\nMert ezen a héten még tavaszi szünet van,..."
2,65,\nMert még nem kezdődött meg a félév...
...,...,...
3,NaN,annak a szaknak a kimondott kolesza ahova felv...
4,NaN,"Nem, nem azt jelenti. A szakkollégium név ar..."
5,NaN,Na de akkor csak abba a koleszba felvételizhe...
0,NaN,"Figyelt kérdés\nLakott már vki ott, és mi..."


# Tiltás elleni védelem

## Usar Agent

-----

**Mi az a user agent?**

A user agent egy olyan információ, amelyet a webböngészők küldenek a weboldalaknak annak érdekében, hogy azok tudják, milyen típusú böngészőt használ a felhasználó. A user agent tartalmazza a böngésző típusát, verzióját, operációs rendszerét, valamint más olyan információkat is, amelyek segíthetnek az oldalaknak abban, hogy a lehető legjobb módon jelenítsék meg a tartalmukat az adott eszközön. A user agent fontos szerepet játszik a webfejlesztés során is, hiszen segítségével a webfejlesztők könnyen tesztelhetik, hogy az adott oldalak megfelelően működnek-e a különböző böngészőkön és eszközökön.

In [484]:
ua = UserAgent()

In [511]:
ua.random

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/46.0.2486.0 Safari/537.36 Edge/13.10586'

## Proxy

**Mi az a proxy?**

A proxy egy olyan szoftver, amely lehetővé teszi, hogy az internetezők anonim módon böngészhessék az internetet, és elrejtsék az IP-címüket. Ez a szoftver közvetítőként működik a felhasználó és az internet között, és megváltoztatja a kérés forrásának azonosítóját, hogy az anonim maradjon. A proxy használata előnyös lehet, ha az internetező biztonságosabb böngészési élményt szeretne, vagy el akarja kerülni a tartalomszűrést, amelyet az internet szolgáltatói vagy más szervezetek végeznek. A proxy segítségével az internetezők szabadon böngészhetnek az interneten, anélkül hogy bárki nyomon követné az online tevékenységüket.

In [512]:
def get_proxies():
    url = "https://free-proxy-list.net/"
    response = requests.get(url)
    soup = bs(response.content, "lxml")
    table = soup.find("div", {"class": "table-responsive"}).table.tbody
    table = table.find_all("tr")
    proxies = [
        (row.find_all("td")[0].text + ":" + row.find_all("td")[1].text) for row in table
    ]
    return [
        {"http": f"http://{proxy}", "https": f"https://{proxy}"} for proxy in proxies
    ]

In [513]:
proxies = get_proxies()

In [514]:
len(proxies)

300

In [303]:
proxies[:5]

[{'http': 'http://202.150.132.53:8080',
  'https': 'https://202.150.132.53:8080'},
 {'http': 'http://80.48.119.28:8080', 'https': 'https://80.48.119.28:8080'},
 {'http': 'http://139.99.237.62:80', 'https': 'https://139.99.237.62:80'},
 {'http': 'http://154.236.189.5:8080', 'https': 'https://154.236.189.5:8080'},
 {'http': 'http://16.163.217.247:8888',
  'https': 'https://16.163.217.247:8888'}]

## Lassítás

In [289]:
sleep(2 + np.random.random())

## Használat egy requestben

In [ ]:
proxy = np.random.choice(proxies)

headers = {"User-Agent": ua.random}

timeout = 3

sleep(2 + np.random.random())

resp = requests.get(URL, headers=headers, proxies=proxy, timeout=3)

In [305]:
def get_soup(
    url: str,
    user_agent: str = None,
    proxy: tuple = None,
    timeout: int = None,
    sleep_time: float = 0,
):
    headers = {"User-Agent": user_agent}
    sleep(sleep_time)
    resp = requests.get(url, headers=headers, proxies=proxy, timeout=timeout)
    resp.raise_for_status()
    return BeautifulSoup(resp.content, "html5lib")

In [319]:
get_soup(URL, user_agent=ua.random, proxy=None, timeout=3, sleep_time=0)

<!DOCTYPE html>
<html lang="hu"><head>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta charset="utf-8"/>
<title>Szakkollégium - Gyakori kérdések</title>
<meta content="Szakkollégium - Kérdések a témában. Pl. A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?" name="description"/>
<meta content="szakkollégium, kérdések, válaszok" name="keywords"/>
<meta content="Szakkollégium - Gyakori kérdések" property="og:title"/>
<meta content="Szakkollégium - Kérdések a témában. Pl. A budapesti szakkollégiumokba csak akkor jelentkezhetek ha az adott egyetemre járok?" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="//static.gyakorikerdesek.hu/p/logo.jpg" property="og:image"/>
<link href="//static.gyakorikerdesek.hu/favicon.ico" rel="shortcut icon"/>
<link href="//static.gyakorikerdesek.hu/apple-touch-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<meta content="#FFFFFF" name="msapplicati

# Adatszerzés PDF-ből

Szöveg kiszedése

In [245]:
text = textract.process("rajk_modell.pdf", method="pdftotext")

Nézzünk bele

In [246]:
text[:100]

b'CHIK\xef\xbf\xbdN ATTILA \xef\xbf\xbd ILY\xef\xbf\xbdS M\xef\xbf\xbdRTON\r\nA Rajk-modell: egy pedag\xef\xbf\xbdgiai k\xef\xbf\xbds\xef\xbf\xbdrlet \xef\xbf\xbds tapasztalata'

In [249]:
text.decode("utf-8")[:100]

'CHIK�N ATTILA � ILY�S M�RTON\r\nA Rajk-modell: egy pedag�giai k�s�rlet �s tapasztalatai\r\nBevezet�s\r\nEg'

Alakítsuk át

In [230]:
pd.Series([text]).str.normalize("NFKD")

0    CHIK�N ATTILA � ILY�S M�RTON\r\nA Rajk-modell:...
dtype: object

Normalizáljuk

In [255]:
normalize("NFKD", text.decode("utf-8"))

'CHIK�N ATTILA � ILY�S M�RTON\r\nA Rajk-modell: egy pedag�giai k�s�rlet �s tapasztalatai\r\nBevezet�s\r\nEgy orsz�g oktat�si rendszer�nek sz�nvonal�t egyebek mellett a l�tez form�k v�ltozatoss�ga, az innovat�v k�s�rletek befogad�sa is m�lyen jellemzi. A szakkoll�giumok val�ban innovat�v form�k: b�r term�szetesen m�s orsz�gokban is tal�lhat�k olyan int�zm�nyek, amelyek a hazai szakkoll�giumok h�rom mark�ns jellemzj�nek, a szakmais�gnak, k�z�ss�gis�gnek �s t�rsadalmi �rz�kenys�gnek ak�r t�bb von�s�val is rendelkeznek, tapasztalataink szerint ezek integr�lt egys�g�nek megval�s�t�sa hungaricum. K�l�n �rdekess�get �s fontoss�got ad a magyar szakkoll�giumi rendszernek, hogy eredeti form�j�ban alulr�l j�v kezdem�nyez�sk�nt j�tt l�tre, s b�r �vtizedek �ta elismerten bizony�t, mind a mai napig l�nyeg�ben javar�szt �llami t�mogat�s n�lk�l mk�dik. Lehet persze, hogy ez haszn�ra v�lt: a hazai felsoktat�s-ir�ny�t�s t�bb �vtizedes cikkcakkjainak k�nyszer k�vet�se val�sz�nleg ink�bb �rtott, mint hasz